I got it working on Colab (with the help of GPT), by adding new pip install commands and removing version numbers from the first pip install. 

DONE, Worked:
* The same should work in my local GPU - TRY OUT!
* this created a cached model under 
  ~/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots
* and ran the local inference on my GPU

https://github.com/alisio/mistral7b-colab-notebook/blob/main/mistral7b_colab.ipynb

# Introduction

In Colab notebook, we will demonstrate how to use the Mistral 7B large language model (LLM) to generate text. The Mistral 7B model is a large-scale transformer-based language model.

This notebook requires the use of a GPU, so be sure to set the correct colab environment if not set already.

## Step 1: Install necessary packages

In this step, we will install the necessary packages for using the Mistral AI language model. This includes the langchain, huggingface-hub, hf_transfer, accelerate, numpy, and pandas packages. We will also install the ctranformers package, which provides an optimized implementation of the transformer architecture used by the Mistral AI model.

In [2]:
%%bash
# install necessary packages
pip install -q langchain huggingface-hub hf_transfer accelerate numpy pandas
#Installs packages conditionally, based on the execution environment
# Check if the system is macOS
if command -v sw_vers &> /dev/null; then
    echo "This script is running on a macOS system."
    CT_METAL=1 pip -q install ctransformers --no-binary ctransformers
else
    # Check if CUDA is installed and if there is a GPU available
    if command -v nvidia-smi &> /dev/null; then
        # If nvidia-smi is present, check if there is a GPU available
        if nvidia-smi -L &> /dev/null; then
            echo "There is a CUDA-enabled GPU available."
            pip install -q ctransformers[cuda]
        else
            echo "CUDA is installed, but no GPU is available."
            pip install -q ctransformers==0.2.27
        fi
    else
        echo "CUDA is not installed or not in PATH."
        pip install -q ctransformers==0.2.27
    fi
fi

There is a CUDA-enabled GPU available.


In [5]:
# in colab: !pip. in jupyterlab: %pip
%pip install -q ctransformers[cuda]

Note: you may need to restart the kernel to use updated packages.


In [6]:
# in colab: !pip. in jupyterlab: %pip
%pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


## Step 2: Prepare the Mistral AI model

In this step, we will prepare the Mistral AI model for use. We will first import the necessary packages and then create an instance of the CTransformers class provided by the langchain_community package. We will then specify the configuration options for the model, including the number of GPU layers to use. Finally, we will prepare the model using the accelerate.prepare method.

In [7]:
from accelerate import Accelerator
from langchain_community.llms import CTransformers
import warnings

warnings.filterwarnings("ignore")


accelerator = Accelerator()

config = {
    "max_new_tokens": 256,
    "repetition_penalty": 1.1,
    "context_length": 3900,
    "temperature": 0,
    "gpu_layers": 50,
}
llm = CTransformers(
    model="TheBloke/Mistral-7B-Instruct-v0.2-GGUF",
    model_file="mistral-7b-instruct-v0.2.Q2_K.gguf",
    model_type="mistral",
    config=config,
)

llm, config = accelerator.prepare(llm, config)

/home/kristjans/projects/hands-on-llm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 1 files: 100%|██████████| 1/1 [02:12<00:00, 132.68s/it]


## Step 3: Generate text using the Mistral AI model

In this step, we will use the Mistral AI model to generate text based on a given prompt. We will use the invoke method provided by the langchain lib to generate text in a streaming fashion. We will specify the prompt as an argument to the invoke method and then print the generated text to the console.

In [8]:
%%time

for text in llm.invoke("Who was ayrton senna?", stream=True):
    print(text, end="", flush=True)



Ayrton Senna (March 3, 1960 – May 1, 1994) was a Brazilian racing driver who is widely regarded as one of the greatest Formula One drivers in history. He won the Formula One World Championship three times (in 1988, 1990, and 1991), and his other achievements include winning the Indianapolis 500 in 1993 and setting numerous track records and pole positions throughout his career. Senna was known for his aggressive driving style, his ability to maximize the performance of his cars, and his intense focus and determination on the race track. He tragically died at the San Marino Grand Prix in 1994.CPU times: user 1min 13s, sys: 1.16 s, total: 1min 14s
Wall time: 20.1 s


# Conclusion

In this notebook, we demonstrated how to use the Mistral AI language model to generate text. We first installed the necessary packages and then prepared the Mistral AI model for use. Finally, we used the model to generate text based on a given prompt.

# Author

* Author: Antonio Alisio de Meneses Cordeiro
* email: alisio.meneses@gmail.com